# Music-2-Notes API Server

Ejecuta la API de Music-2-Notes en Google Colab con GPU gratis y la expone via ngrok.

**Requisitos:**
- Runtime con GPU: `Runtime > Change runtime type > T4 GPU`
- Cuenta gratuita en [ngrok](https://ngrok.com) para obtener tu authtoken

## 1. Configuracion

In [ ]:
#@title Configura tu ngrok authtoken { display-mode: "form" }
#@markdown Obtenlo gratis en https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTHTOKEN = "" #@param {type:"string"}
MODEL_SIZE = "tiny" #@param ["tiny", "full"]
CONFIDENCE_THRESHOLD = 0.5 #@param {type:"number"}

## 2. Instalar dependencias

In [ ]:
%%capture
# Colab ya tiene torch+CUDA, solo instalamos lo que falta
!pip install fastapi uvicorn[standard] python-multipart pydantic pydantic-settings \
    sqlalchemy aiosqlite librosa torchcrepe soundfile mido httpx aiofiles pyngrok scipy

## 3. Clonar repositorio

In [ ]:
import os

REPO_URL = "https://github.com/yarango582/music-2-notes.git"
REPO_DIR = "/content/music-2-notes"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL} {REPO_DIR}
else:
    !cd {REPO_DIR} && git pull

os.chdir(REPO_DIR)
print(f"Directorio: {os.getcwd()}")

## 4. Configurar entorno

In [ ]:
import torch

# Crear carpetas necesarias
os.makedirs("data", exist_ok=True)
os.makedirs("storage", exist_ok=True)

# Configurar variables de entorno
os.environ["ENV"] = "production"
os.environ["DATABASE_URL"] = "sqlite+aiosqlite:///data/music2notes.db"
os.environ["STORAGE_PATH"] = os.path.join(REPO_DIR, "storage")
os.environ["DEFAULT_MODEL_SIZE"] = MODEL_SIZE
os.environ["DEFAULT_CONFIDENCE_THRESHOLD"] = str(CONFIDENCE_THRESHOLD)
os.environ["SECRET_KEY"] = "colab-session-key"
os.environ["WEBHOOK_SECRET_KEY"] = "colab-webhook-key"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"PyTorch device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")
else:
    print("ADVERTENCIA: No hay GPU. Cambia el runtime a T4 GPU para mejor rendimiento.")

## 5. Iniciar API con ngrok

In [ ]:
import sys
import threading
import time
import uvicorn
from pyngrok import ngrok, conf
from IPython.display import display, HTML

# Agregar el repo al path para imports
if REPO_DIR not in sys.path:
    sys.path.insert(0, REPO_DIR)

# Configurar ngrok
assert NGROK_AUTHTOKEN, "Configura tu NGROK_AUTHTOKEN en la celda 1"
conf.get_default().auth_token = NGROK_AUTHTOKEN

PORT = 8000

# Iniciar uvicorn en un thread
def run_server():
    uvicorn.run(
        "src.api.main:app",
        host="0.0.0.0",
        port=PORT,
        log_level="info",
    )

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(3)  # Esperar que arranque

# Abrir tunel ngrok
public_url = ngrok.connect(PORT, "http").public_url

display(HTML(f"""
<div style="padding:20px; background:#1a1a2e; border-radius:12px; margin:10px 0; font-family:monospace;">
  <h2 style="color:#e94560; margin:0 0 15px 0;">Music-2-Notes API</h2>
  <p style="color:#eee; font-size:16px;">URL publica: <a href="{public_url}" target="_blank" style="color:#0f3460;">{public_url}</a></p>
  <p style="color:#eee; font-size:16px;">Docs (Swagger): <a href="{public_url}/docs" target="_blank" style="color:#0f3460;">{public_url}/docs</a></p>
  <p style="color:#eee; font-size:16px;">Health: <a href="{public_url}/api/v1/health" target="_blank" style="color:#0f3460;">{public_url}/api/v1/health</a></p>
  <hr style="border-color:#333;">
  <p style="color:#aaa; font-size:13px;">GPU: {device.upper()} | Modelo: {MODEL_SIZE} | Confianza: {CONFIDENCE_THRESHOLD}</p>
</div>
"""))

print(f"\nEjemplo de uso con curl:")
print(f'curl -X POST "{public_url}/api/v1/jobs" -F "audio_file=@tu_audio.wav"')

## 6. Probar la API

In [ ]:
import numpy as np
import soundfile as sf
import httpx
import json

# Generar audio de prueba: nota A4 (440Hz) por 2 segundos
sr = 16000
t = np.linspace(0, 2.0, sr * 2, endpoint=False)
audio = 0.5 * np.sin(2 * np.pi * 440 * t)
test_path = "/tmp/test_a4.wav"
sf.write(test_path, audio, sr)

# Enviar al API
print("Enviando audio de prueba (A4, 440Hz, 2s)...")
with open(test_path, "rb") as f:
    r = httpx.post(
        f"http://localhost:{PORT}/api/v1/jobs",
        files={"audio_file": ("test_a4.wav", f, "audio/wav")},
        data={"model_size": MODEL_SIZE, "confidence_threshold": str(CONFIDENCE_THRESHOLD)},
    )

job = r.json()
job_id = job["job_id"]
print(f"Job creado: {job_id}")

# Esperar resultado
print("Procesando", end="")
for _ in range(120):
    time.sleep(2)
    status = httpx.get(f"http://localhost:{PORT}/api/v1/jobs/{job_id}").json()
    if status["status"] == "completed":
        print(f" OK! ({status['processing_time']:.1f}s)")
        break
    elif status["status"] == "failed":
        print(f" ERROR: {status['error_message']}")
        break
    print(".", end="", flush=True)
else:
    print(" TIMEOUT")

# Mostrar resultado
if status["status"] == "completed":
    result = httpx.get(f"http://localhost:{PORT}/api/v1/jobs/{job_id}/result").json()
    notes = result["result"]["notes"]
    print(f"\nNotas detectadas: {len(notes)}")
    for n in notes[:10]:
        print(f"  {n['note_name']:4s} | {n['start_time']:.2f}s | dur: {n['duration']:.2f}s | freq: {n['frequency']:.1f}Hz | conf: {n['confidence']:.3f}")

## 7. Subir tu propio audio

In [ ]:
from google.colab import files

print("Selecciona un archivo de audio (WAV, MP3, FLAC, OGG, M4A):")
uploaded = files.upload()

for filename, content in uploaded.items():
    print(f"\nProcesando: {filename}")

    r = httpx.post(
        f"http://localhost:{PORT}/api/v1/jobs",
        files={"audio_file": (filename, content, "audio/wav")},
        data={"model_size": MODEL_SIZE, "confidence_threshold": str(CONFIDENCE_THRESHOLD)},
    )
    job_id = r.json()["job_id"]
    print(f"Job: {job_id}")

    print("Procesando", end="")
    for _ in range(300):
        time.sleep(2)
        s = httpx.get(f"http://localhost:{PORT}/api/v1/jobs/{job_id}").json()
        if s["status"] == "completed":
            print(f" OK! ({s['processing_time']:.1f}s)")
            break
        elif s["status"] == "failed":
            print(f" ERROR: {s['error_message']}")
            break
        print(".", end="", flush=True)

    if s["status"] == "completed":
        # Descargar MIDI
        midi = httpx.get(f"http://localhost:{PORT}/api/v1/jobs/{job_id}/download/midi")
        midi_name = filename.rsplit('.', 1)[0] + ".mid"
        with open(midi_name, "wb") as f:
            f.write(midi.content)

        # Descargar JSON
        result_json = httpx.get(f"http://localhost:{PORT}/api/v1/jobs/{job_id}/download/json")
        json_name = filename.rsplit('.', 1)[0] + ".json"
        with open(json_name, "wb") as f:
            f.write(result_json.content)

        result = httpx.get(f"http://localhost:{PORT}/api/v1/jobs/{job_id}/result").json()
        notes = result["result"]["notes"]
        print(f"Notas detectadas: {len(notes)}")
        print(f"Duracion audio: {s['audio_duration']:.1f}s")

        # Descargar archivos generados
        print(f"\nDescargando {midi_name} y {json_name}...")
        files.download(midi_name)
        files.download(json_name)

---

**Nota:** La sesion de Colab y el tunel ngrok se mantienen activos mientras esta pestana este abierta. Cualquier aplicacion externa puede usar la URL publica de ngrok para enviar audios via la API.